In [2]:
import pandas as pd
import numpy as np

parados = pd.read_csv("ine_parados_educacion_ccaa_2014_2024.csv", sep = ";", encoding = "latin-1")
activos = pd.read_csv("ine_activos_educacion_ccaa_2014_2024.csv", sep = ";", encoding = "latin-1")

In [3]:
parados.info()
activos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1463 entries, 0 to 1462
Data columns (total 5 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Sexo                              1463 non-null   object
 1   Comunidades y Ciudades Autónomas  1463 non-null   object
 2   Nivel de formación alcanzado      1463 non-null   object
 3   Periodo                           1463 non-null   int64 
 4   Total                             1463 non-null   object
dtypes: int64(1), object(4)
memory usage: 57.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1463 entries, 0 to 1462
Data columns (total 5 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Sexo                              1463 non-null   object
 1   Comunidades y Ciudades Autónomas  1463 non-null   object
 2   Nivel de formación alcanzado      1463 non-null   

In [4]:
#Eliminar columna sexo:
parados = parados.drop(columns=["Sexo"])
activos = activos.drop(columns=["Sexo"])

In [5]:
#renombrar columnas
rename_columns = {
    "Comunidades y Ciudades Autónomas": "comunidad",
    "Nivel de formación alcanzado": "nivel_educativo",
    "Periodo": "ano",
    "Total": "miles"}

parados = parados.rename(columns=rename_columns)
activos = activos.rename(columns=rename_columns)

In [6]:
#reclasificar nivel educativo
parados["nivel_educativo"].unique() #para saber categorías actuales

def clasificar_educacion(nivel):
    if nivel == "Analfabetos":
        return "Analfabetos"
    elif nivel in ["Estudios primarios incompletos", "Educación primaria"]:
        return "Básica"
    elif "Primera etapa de Educación Secundaria y similar" in nivel:
        return "ESO"
    elif "Segunda etapa de educación secundaria con orientación general" in nivel:
        return "Bachillerato"
    elif "Segunda etapa de educación secundaria con orientación profesional (incluye educación postsecundaria no superior)" in nivel:
        return "FP"
    elif "Educación superior" in nivel:
        return "Superior"
    else:
        return "Otros"

parados["nivel_educativo"] = parados["nivel_educativo"].apply(clasificar_educacion)
activos["nivel_educativo"] = activos["nivel_educativo"].apply(clasificar_educacion)

In [7]:
#convertir columna valor a número ya que actualmente es un objeto.
parados["miles"] = (parados["miles"].astype(str).str.replace(",", ".", regex=False))
activos["miles"] = (activos["miles"].astype(str).str.replace(",", ".", regex=False))

parados["miles"] = pd.to_numeric(parados["miles"], errors="coerce")
activos["miles"] = pd.to_numeric(activos["miles"], errors="coerce")

In [8]:
#unir activos y parados en una base de datos.
activos_parados_ccaa_2014_2024 = pd.merge(parados, activos, on=["comunidad", "nivel_educativo", "ano"], suffixes=("_parados", "_activos"))

activos_parados_ccaa_2014_2024

,comunidad,nivel_educativo,ano,miles_parados,miles_activos
0,01 Andalucía,Analfabetos,2014,18.3,27.3
1,01 Andalucía,Analfabetos,2015,14.2,22.9
2,01 Andalucía,Analfabetos,2016,10.2,20.7
3,01 Andalucía,Analfabetos,2017,7.2,19.8
4,01 Andalucía,Analfabetos,2018,5.8,15.0
...,...,...,...,...,...
1876,19 Melilla,Superior,2020,1.1,15.1
1877,19 Melilla,Superior,2021,1.7,15.2
1878,19 Melilla,Superior,2022,1.8,14.4
1879,19 Melilla,Superior,2023,1.5,13.9


In [9]:
#eliminar números de delante la comunidad
activos_parados_ccaa_2014_2024["comunidad"] = activos_parados_ccaa_2014_2024["comunidad"].str.replace(r"^\d+\s+", "", regex=True)

In [10]:
#calcular tasa de paro
activos_parados_ccaa_2014_2024["tasa_paro"] = round((activos_parados_ccaa_2014_2024["miles_parados"] / activos_parados_ccaa_2014_2024["miles_activos"]) * 100, 2)

In [11]:
#ordenar datos y añadir index
activos_parados_ccaa_2014_2024.sort_values(by=["ano", "comunidad", "nivel_educativo"]).reset_index(drop=True)

activos_parados_ccaa_2014_2024

,comunidad,nivel_educativo,ano,miles_parados,miles_activos,tasa_paro
0,Andalucía,Analfabetos,2014,18.3,27.3,67.03
1,Andalucía,Analfabetos,2015,14.2,22.9,62.01
2,Andalucía,Analfabetos,2016,10.2,20.7,49.28
3,Andalucía,Analfabetos,2017,7.2,19.8,36.36
4,Andalucía,Analfabetos,2018,5.8,15.0,38.67
...,...,...,...,...,...,...
1876,Melilla,Superior,2020,1.1,15.1,7.28
1877,Melilla,Superior,2021,1.7,15.2,11.18
1878,Melilla,Superior,2022,1.8,14.4,12.50
1879,Melilla,Superior,2023,1.5,13.9,10.79


In [12]:
#guardar nueva base de datos
activos_parados_ccaa_2014_2024.to_csv("paro_educacion_ccaa_2014_2024.csv", index=False)

## Resultado de la fase de limpieza y preparación de datos

Se ha construido un dataset final a nivel de:
- Comunidad Autónoma
- Año (2014–2024)
- Nivel educativo (6 categorías)

Variables finales:
- miles_parados: personas desempleadas (en miles)
- miles_activos: población activa (en miles)
- tasa_paro: porcentaje de desempleo

Los valores absolutos se mantienen en miles de personas para preservar la escala original del INE.
Este dataset se utilizará como base para el análisis exploratorio y el dashboard.